In [1]:
import pandas as pd
import glob
import time

path = '../data/raw/*.csv'
csv_files = glob.glob(path)
df = []

for file in csv_files:
    df = pd.read_csv(file)
    print(list(df))

['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']


In [12]:
def shift_column(df, col):
    col_list = list(df)
    #col1_index = df.columns.get_loc(col1) #or list(df.columns).index(col1)
    #col2_index = df.columns.get_loc(col2)
    #col_list[col1_index], col_list[col2_index] = col_list[col2_index], col_list[col1_index]
    #df.columns = col_list
    col1 = df.pop(col)
    df.insert(len(col_list) - 1, col, col1)

def numbers_only(string):
    year_index = string.index(' y')
    year = string[:year_index]
    duration = 0
    if ' months' in string:
        month_index_start = string.index('s ') + 2
        month_index_end = string.index(' m')
        month = string[month_index_start:month_index_end]
        month_adjusted = str(float(month) / 12)
        duration = float(year + month_adjusted)
    else:
        duration = year
    return duration 

i = 0

for file in csv_files:
    i += 1
    df = pd.read_csv(file)
    if 'remaining_lease' in df.columns:
        shift_column(df, 'remaining_lease') #idt need to define a separate function
        #print(df1[:5])
     #df = pd.concat([df, df1]) #I realised I forgot to exclude header when concatenating...

#print(df[:5])
        
    print(len(df))
    df.drop_duplicates()
    print(len(df))
    
    start = time.time()
    df['month'] = df['month'].apply(lambda x: pd.to_datetime(x + '-01'))
    df.sort_values(by='month')

    elapsed = time.time() - start
    print(f'took {elapsed} seconds to run')

    if 'remaining_lease' in df.columns:
        try:
            df['remaining_lease'] = df['remaining_lease'].astype('int32')
        except ValueError:
            df['remaining_lease'] = df['remaining_lease'].apply(lambda x: numbers_only(x) if type(x) == str else None)

    for column in df.columns:
        exclude = ['month', 'floor_area_sqm', 'lease_commence_date', 'resale_price', 'remaining_lease']
        if column not in exclude:
            df[column] = df[column].str.replace('-', ' ').str.lower()
            df[column] = df[column].astype('category')

    i_string = str(i)
    df.to_csv(f'../data/cleaned data {i_string}.csv', index=False)

52203
52203
took 29.654247999191284 seconds to run
369651
369651
took 210.70340490341187 seconds to run
195062
195062
took 115.0842809677124 seconds to run
37153
37153
took 21.268896102905273 seconds to run
287196
287196
took 168.6399405002594 seconds to run


In [ ]:
import psycopg2
import yaml

# Accessing the credentials
def load_config():
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
    return config

config = load_config()

# Now you can use the credentials to connect to the database
database_config = config['database']
dbname = database_config['name']
user = database_config['user']
password = database_config['password']
host = database_config['host']

#print(f"Connecting to {dbname} at {host} with user {user}")

# Create a connection to PostgreSQL
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)

# Create a cursor object
cursor = conn.cursor()

########################################################################################## 

# Create the table if it doesn't already exist
create_table_query = """
CREATE TABLE IF NOT EXISTS flats (
    flat_id SERIAL PRIMARY KEY,
    commence_date INTEGER,
    remaining_lease FLOAT,
    town VARCHAR(100),
    street_name VARCHAR(255),
    block VARCHAR(10),
    flat_type VARCHAR(50),
    flat_model VARCHAR(50),
    storey_range VARCHAR(50),
    floor_area_sqm FLOAT
);

CREATE TABLE IF NOT EXISTS lease (
    lease_id SERIAL PRIMARY KEY,
    flat_id INTEGER,
    month TIMESTAMP,
    resale_price FLOAT,
    CONSTRAINT fk_flats FOREIGN KEY (flat_id) REFERENCES flats(flat_id)
);
"""
cursor.execute(create_table_query)
conn.commit()  # Commit the creation of the table

# Load the CSV data into a Pandas DataFrame
path1 = '../data/*.csv'
csvfiles = glob.glob(path1)
df1 = []

for file in csvfiles:
    df1 = pd.read_csv(file)

    # Insert data from DataFrame into the PostgreSQL table
    for index, row in df1.iterrows():
        cursor.execute("""
        INSERT INTO flats (commence_date, town, street_name, block, flat_type, flat_model, storey_range, floor_area_sqm)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (row['lease_commence_date'], row['town'], row['street_name'], row['block'], row['flat_type'], row['flat_model'], 
              row['storey_range'], row['floor_area_sqm']))

        if 'remaining_lease' in df1.columns:
            cursor.execute("""
            INSERT INTO flats (remaining_lease)
            VALUES (%s)
            """, ([row['remaining_lease']]))

        cursor.execute("""
        SELECT flat_id FROM flats WHERE town = %s AND street_name = %s
        """, (row['town'], row['street_name']))

        # Fetch the flat_id value
        flat_id = cursor.fetchone()
        
        cursor.execute("""
        INSERT INTO lease (flat_id, month, resale_price)
        VALUES (%s, %s, %s)
        """, (flat_id, row['month'], row['resale_price']))

# Commit the transaction
conn.commit()

##########################################################################################

# Close the cursor and connection
cursor.close()
conn.close()